### Plotting for single filteration iteration

In [150]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import itertools
from itertools import combinations
from scipy import spatial
import pickle as pickle
import gudhi
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import io
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageChops, ImageFont
import shapely.geometry as geom
from shapely.ops import unary_union
import warnings

import invr

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [151]:
#first create the grid system

In [152]:
# Create a DataFrame with box names and values
box_names = [chr(65 + i) for i in range(16)]
values = list(range(0, 16))
values_1 = [1,2,3,4,8,7,6,5,9,10,11,12,16,15,14,13]
values_2 = list(range(1, 17))
df = pd.DataFrame({'FIPS': box_names, 'Value': values,'Value_1': values_1,'Value_2': values_2})

In [153]:
df

,FIPS,Value,Value_1,Value_2
0,A,0,1,1
1,B,1,2,2
2,C,2,3,3
3,D,3,4,4
4,E,4,8,5
5,F,5,7,6
6,G,6,6,7
7,H,7,5,8
8,I,8,9,9
9,J,9,10,10


In [154]:
# Sorting the DataFrame based on the 'rate' column
df.sort_values(by='Value_2', inplace=True)

In [155]:
# Adding a new column 'new_ID' with ID values starting from zero
df['sortedID'] = range(len(df))

In [156]:
df = df[['FIPS','Value', 'sortedID', 'Value_2']]

In [157]:
# Function to calculate square coordinates for a given name
def calculate_square_coordinates(row):
    box_name = row['FIPS']
    value = row['Value']
    # Assuming each smaller square has a side length of 1 unit
    x = value % 4
    y = value // 4
    # Return square coordinates as a Shapely polygon
    return geom.Polygon([(x, y), (x+1, y), (x+1, y+1), (x, y+1)])

In [158]:
# Calculate square coordinates for each row and create a geometry column
df['geometry'] = df.apply(calculate_square_coordinates, axis=1)

In [159]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [160]:
# Set the CRS to a simple Cartesian coordinate system
gdf.crs = "EPSG:3395"  # This is a commonly used projected CRS

In [161]:
gdf

,FIPS,Value,sortedID,Value_2,geometry
0,A,0,0,1,"POLYGON ((0.000 0.000, 1.000 0.000, 1.000 1.00..."
1,B,1,1,2,"POLYGON ((1.000 0.000, 2.000 0.000, 2.000 1.00..."
2,C,2,2,3,"POLYGON ((2.000 0.000, 3.000 0.000, 3.000 1.00..."
3,D,3,3,4,"POLYGON ((3.000 0.000, 4.000 0.000, 4.000 1.00..."
4,E,4,4,5,"POLYGON ((0.000 1.000, 1.000 1.000, 1.000 2.00..."
5,F,5,5,6,"POLYGON ((1.000 1.000, 2.000 1.000, 2.000 2.00..."
6,G,6,6,7,"POLYGON ((2.000 1.000, 3.000 1.000, 3.000 2.00..."
7,H,7,7,8,"POLYGON ((3.000 1.000, 4.000 1.000, 4.000 2.00..."
8,I,8,8,9,"POLYGON ((0.000 2.000, 1.000 2.000, 1.000 3.00..."
9,J,9,9,10,"POLYGON ((1.000 2.000, 2.000 2.000, 2.000 3.00..."


In [162]:
gdf.sort_values(by='Value', inplace=True)

In [163]:
gdf

,FIPS,Value,sortedID,Value_2,geometry
0,A,0,0,1,"POLYGON ((0.000 0.000, 1.000 0.000, 1.000 1.00..."
1,B,1,1,2,"POLYGON ((1.000 0.000, 2.000 0.000, 2.000 1.00..."
2,C,2,2,3,"POLYGON ((2.000 0.000, 3.000 0.000, 3.000 1.00..."
3,D,3,3,4,"POLYGON ((3.000 0.000, 4.000 0.000, 4.000 1.00..."
4,E,4,4,5,"POLYGON ((0.000 1.000, 1.000 1.000, 1.000 2.00..."
5,F,5,5,6,"POLYGON ((1.000 1.000, 2.000 1.000, 2.000 2.00..."
6,G,6,6,7,"POLYGON ((2.000 1.000, 3.000 1.000, 3.000 2.00..."
7,H,7,7,8,"POLYGON ((3.000 1.000, 4.000 1.000, 4.000 2.00..."
8,I,8,8,9,"POLYGON ((0.000 2.000, 1.000 2.000, 1.000 3.00..."
9,J,9,9,10,"POLYGON ((1.000 2.000, 2.000 2.000, 2.000 3.00..."


In [187]:
#______________

In [164]:
# Now let's apply fileration level and then create the adjacent counties for each county

In [165]:
filtration_threshold_ = 14
filtered_df = gdf[gdf['Value_2'] < filtration_threshold_]

In [166]:
filtered_df

,FIPS,Value,sortedID,Value_2,geometry
0,A,0,0,1,"POLYGON ((0.000 0.000, 1.000 0.000, 1.000 1.00..."
1,B,1,1,2,"POLYGON ((1.000 0.000, 2.000 0.000, 2.000 1.00..."
2,C,2,2,3,"POLYGON ((2.000 0.000, 3.000 0.000, 3.000 1.00..."
3,D,3,3,4,"POLYGON ((3.000 0.000, 4.000 0.000, 4.000 1.00..."
4,E,4,4,5,"POLYGON ((0.000 1.000, 1.000 1.000, 1.000 2.00..."
5,F,5,5,6,"POLYGON ((1.000 1.000, 2.000 1.000, 2.000 2.00..."
6,G,6,6,7,"POLYGON ((2.000 1.000, 3.000 1.000, 3.000 2.00..."
7,H,7,7,8,"POLYGON ((3.000 1.000, 4.000 1.000, 4.000 2.00..."
8,I,8,8,9,"POLYGON ((0.000 2.000, 1.000 2.000, 1.000 3.00..."
9,J,9,9,10,"POLYGON ((1.000 2.000, 2.000 2.000, 2.000 3.00..."


In [167]:
# Perform a spatial join to find adjacent precincts
adjacent_precincts = gpd.sjoin(filtered_df, filtered_df, predicate='intersects', how='left')

 # Filter the results to include only the adjacent states
adjacent_precincts = adjacent_precincts.query('sortedID_left != sortedID_right')

# Group the resulting dataframe by the original precinct Name and create a list of adjacent precinct Name
adjacent_precincts = adjacent_precincts.groupby('sortedID_left')['sortedID_right'].apply(list).reset_index()

In [168]:
adjacent_precincts.rename(columns={'sortedID_left': 'county', 'sortedID_right': 'adjacent'}, inplace=True)

In [169]:
adjacent_precincts

,county,adjacent
0,0,"[1, 4, 5]"
1,1,"[2, 6, 0, 4, 5]"
2,2,"[3, 6, 7, 1, 5]"
3,3,"[2, 6, 7]"
4,4,"[0, 1, 5, 8, 9]"
5,5,"[2, 6, 10, 0, 1, 4, 8, 9]"
6,6,"[2, 3, 7, 10, 11, 1, 5, 9]"
7,7,"[2, 3, 6, 10, 11]"
8,8,"[4, 5, 9, 12]"
9,9,"[6, 10, 4, 5, 8, 12]"


In [170]:
adjacencies = adjacent_precincts['adjacent'].tolist()

In [171]:
adjacencies

[[1, 4, 5],
 [2, 6, 0, 4, 5],
 [3, 6, 7, 1, 5],
 [2, 6, 7],
 [0, 1, 5, 8, 9],
 [2, 6, 10, 0, 1, 4, 8, 9],
 [2, 3, 7, 10, 11, 1, 5, 9],
 [2, 3, 6, 10, 11],
 [4, 5, 9, 12],
 [6, 10, 4, 5, 8, 12],
 [6, 7, 11, 5, 9],
 [6, 7, 10],
 [8, 9]]

In [172]:
merged_df = pd.merge(adjacent_precincts, gdf, left_on='county',right_on='sortedID', how='left')

In [173]:
gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')

In [174]:
gdf

,county,adjacent,FIPS,Value,sortedID,Value_2,geometry
0,0,"[1, 4, 5]",A,0,0,1,"POLYGON ((0.000 0.000, 1.000 0.000, 1.000 1.00..."
1,1,"[2, 6, 0, 4, 5]",B,1,1,2,"POLYGON ((1.000 0.000, 2.000 0.000, 2.000 1.00..."
2,2,"[3, 6, 7, 1, 5]",C,2,2,3,"POLYGON ((2.000 0.000, 3.000 0.000, 3.000 1.00..."
3,3,"[2, 6, 7]",D,3,3,4,"POLYGON ((3.000 0.000, 4.000 0.000, 4.000 1.00..."
4,4,"[0, 1, 5, 8, 9]",E,4,4,5,"POLYGON ((0.000 1.000, 1.000 1.000, 1.000 2.00..."
5,5,"[2, 6, 10, 0, 1, 4, 8, 9]",F,5,5,6,"POLYGON ((1.000 1.000, 2.000 1.000, 2.000 2.00..."
6,6,"[2, 3, 7, 10, 11, 1, 5, 9]",G,6,6,7,"POLYGON ((2.000 1.000, 3.000 1.000, 3.000 2.00..."
7,7,"[2, 3, 6, 10, 11]",H,7,7,8,"POLYGON ((3.000 1.000, 4.000 1.000, 4.000 2.00..."
8,8,"[4, 5, 9, 12]",I,8,8,9,"POLYGON ((0.000 2.000, 1.000 2.000, 1.000 3.00..."
9,9,"[6, 10, 4, 5, 8, 12]",J,9,9,10,"POLYGON ((1.000 2.000, 2.000 2.000, 2.000 3.00..."


In [175]:
maxDimension = 3

In [176]:
adjacencies

[[1, 4, 5],
 [2, 6, 0, 4, 5],
 [3, 6, 7, 1, 5],
 [2, 6, 7],
 [0, 1, 5, 8, 9],
 [2, 6, 10, 0, 1, 4, 8, 9],
 [2, 3, 7, 10, 11, 1, 5, 9],
 [2, 3, 6, 10, 11],
 [4, 5, 9, 12],
 [6, 10, 4, 5, 8, 12],
 [6, 7, 11, 5, 9],
 [6, 7, 10],
 [8, 9]]

In [177]:
preferences = np.asarray(merged_df['Value_2'].tolist())

In [178]:
preferences

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [179]:
V = []
V = invr.incremental_vr(V, adjacencies, maxDimension)

In [180]:
V

[[0],
 [1],
 [0, 1],
 [2],
 [1, 2],
 [3],
 [2, 3],
 [4],
 [0, 4],
 [1, 4],
 [0, 1, 4],
 [5],
 [0, 5],
 [1, 5],
 [0, 1, 5],
 [2, 5],
 [1, 2, 5],
 [4, 5],
 [0, 4, 5],
 [1, 4, 5],
 [6],
 [1, 6],
 [2, 6],
 [1, 2, 6],
 [3, 6],
 [2, 3, 6],
 [5, 6],
 [1, 5, 6],
 [2, 5, 6],
 [7],
 [2, 7],
 [3, 7],
 [2, 3, 7],
 [6, 7],
 [2, 6, 7],
 [3, 6, 7],
 [8],
 [4, 8],
 [5, 8],
 [4, 5, 8],
 [9],
 [4, 9],
 [5, 9],
 [4, 5, 9],
 [6, 9],
 [5, 6, 9],
 [8, 9],
 [4, 8, 9],
 [5, 8, 9],
 [10],
 [5, 10],
 [6, 10],
 [5, 6, 10],
 [7, 10],
 [6, 7, 10],
 [9, 10],
 [5, 9, 10],
 [6, 9, 10],
 [11],
 [6, 11],
 [7, 11],
 [6, 7, 11],
 [10, 11],
 [6, 10, 11],
 [7, 10, 11],
 [12],
 [8, 12],
 [9, 12],
 [8, 9, 12]]

In [181]:
# Plotting part

In [182]:
#city centroids
city_coordinates = {city.sortedID: np.array((city.geometry.centroid.x, city.geometry.centroid.y)) for _, city in gdf.iterrows()}

In [183]:
city_coordinates

{0: array([0.5, 0.5]),
 1: array([1.5, 0.5]),
 2: array([2.5, 0.5]),
 3: array([3.5, 0.5]),
 4: array([0.5, 1.5]),
 5: array([1.5, 1.5]),
 6: array([2.5, 1.5]),
 7: array([3.5, 1.5]),
 8: array([0.5, 2.5]),
 9: array([1.5, 2.5]),
 10: array([2.5, 2.5]),
 11: array([3.5, 2.5]),
 12: array([0.5, 3.5])}

In [184]:
def fig2img(fig):
     #convert matplot fig to image and return it

     buf = io.BytesIO()
     fig.savefig(buf)
     buf.seek(0)
     img = Image.open(buf)
     return img

In [185]:
list_gif = []

# Create a figure and axis
fig, ax = plt.subplots(figsize=(20, 20))
ax.set_axis_off() 

# Plot the "wyoming_svi" DataFrame
gdf.plot(ax=ax, edgecolor='black', linewidth=0.3, color="white")

# Plot the centroid of the large square with values
for i, row in gdf.iterrows():
    centroid = row['geometry'].centroid
    plt.text(centroid.x, centroid.y, str(row['FIPS']), fontsize=15, ha='center', color="black")

for edge_or_traingle in V:

    
    if len(edge_or_traingle) == 2:
        # Plot an edge
        ax.plot(*zip(*[city_coordinates[vertex] for vertex in edge_or_traingle]), color='red', linewidth=2)
        img = fig2img(fig)
        list_gif.append(img)
    elif len(edge_or_traingle) == 3:
        # Plot a triangle
        ax.add_patch(plt.Polygon([city_coordinates[vertex] for vertex in edge_or_traingle], color='green', alpha=0.2))
        img = fig2img(fig)
        list_gif.append(img)
plt.close()




In [186]:
list_gif[0].save('value_1.gif',
                 save_all=True,append_images=list_gif[1:],optimize=False,duration=1000,loop=0)